# Задача

Аналитику необходимо получить следующую информацию: в каких ресторанах
KFC он может позавтракать (имеется в виду именно завтрак KFC) в Новосибирске
в 8:45.

Мы понимаем, что в дальнейшем могут появится новые задачи по этой теме и
решаем собрать данные по ресторанам KFC в БД, а аналитику отдать SQL-запрос
для получения нужной ему информации.

## Таким образом, задача заключается в следующем:
- скачать с сайта KFC информацию о ресторанах
- сложить эту информацию в базу данных SQLite
- написать запрос для получения информации, которая необходима аналитику

## Что ожидаем получить на выходе:
- работающий код решения на Python или R
- файл с базой данных SQLite с информацией о ресторанах KFC
- соответствующий задаче SQL-запрос к этой базе данных

## Примечания:
- для решения задачи не нужно парсить страницы сайта
- если не получается решить задачу без парсинга, можно получить подсказку, откуда именно нужно скачать данные
- не обязательно складывать в базу всю информацию, которая есть на сайте, достаточно выбрать только основные поля, необходимые для выполнения задания (+координаты и адрес)

## Загрузим данные о ресторанах в базу данных sqlite:

In [1]:
import requests

import sqlite3 as sl

import pandas as pd

# Ссылка с методом для получения информации:

url = 'https://api.kfc.com/api/store/v2/store.get_restaurants'

# Загрузим данные о ресторанах:

response = requests.get(url)

data = response.json()

# Определим количество ресторанов:

restorans_count = len(data['searchResults'])

# Создадим базу данных и связь с ней:

conn = sl.connect("kfc.db")

# Создадим объект класса cursor для взаимодействия с базой данных:

cursor = conn.cursor()

# Удалим неактуальную таблицу, если такая есть:

cursor.execute("""DROP TABLE IF EXISTS restaurants""")

# Создадим таблицу для записи данных:

cursor.execute("""CREATE TABLE restaurants
                  (city TEXT, address TEXT, latitude FLOAT,
                   longitude FLOAT, start_time TEXT, end_time TEXT, 
                   breakfast_start_time TEXT, breakfast_end_time TEXT)
               """)

# Произведём запись полученых с сайта данных в sqlite: 

for i in range(restorans_count):
    
    city = data['searchResults'][i]['storePublic']['contacts']['city']['ru']
    
    try:
    
        address = data['searchResults'][i]['storePublic']['contacts']['streetAddress']['ru']
        
    except LookupError:
        
        address = ''
    
    latitude = data['searchResults'][i]['storePublic']['contacts']['coordinates']['geometry']['coordinates'][0]
    
    longitude = data['searchResults'][i]['storePublic']['contacts']['coordinates']['geometry']['coordinates'][1]
    
    start_time = data['searchResults'][i]['storePublic']['openingHours']['regular']['startTimeLocal']
    
    end_time = data['searchResults'][i]['storePublic']['openingHours']['regular']['endTimeLocal']
    
    breakfast_start_time = data['searchResults'][i]['storePublic']['menues'][0]['availability']['regular']['startTimeLocal']
    
    breakfast_end_time = data['searchResults'][i]['storePublic']['menues'][0]['availability']['regular']['endTimeLocal']
    
    # Добавим строки, которые имеют адрес и непустое значение параметра 'start_time':
    
    if address != '':
        
        if i > 3:
    
            cursor.execute("""INSERT INTO restaurants VALUES (?, ?, ?, ?, ?, ?, ?, ?)""",
                           (city, address, latitude,
                            longitude, start_time, end_time, 
                            breakfast_start_time, breakfast_end_time))

restaurants_df = pd.read_sql('select * from restaurants', conn)

# Сохраним данные:

conn.commit()

# Закроем соединение:
    
conn.close()

## Просмотрим содержимое базы данных:

In [2]:
restaurants_df

,city,address,latitude,longitude,start_time,end_time,breakfast_start_time,breakfast_end_time
0,Санкт-Петербург,"191025, Санкт-Петербург, Маяковского, 1/96",59.932126,30.354398,08:00:00,07:00:00,None,None
1,Санкт-Петербург,"194356, Санкт-Петербург, Выборгское шоссе, 3к1",60.037834,30.320718,08:00:00,23:00:00,09:00:00,11:00:00
2,Москва и МО,"125445, Москва, Правобережная, 1",55.880997,37.449501,10:00:00,20:45:00,None,None
3,Москва и МО,"142704, Москва, Калужское шоссе, 21-й километр",55.604435,37.490441,10:00:00,21:45:00,09:00:00,11:00:00
4,Санкт-Петербург,"195112, Санкт-Петербург, Заневский проспект, 67к2",59.932910,30.437677,10:00:00,21:45:00,10:00:00,11:00:00
...,...,...,...,...,...,...,...,...
1047,Абакан,"655010, Абакан, Торговая, дом 5",53.742783,91.435138,08:00:00,21:00:00,09:00:00,11:00:00
1048,Самара,"443028, Самара, Мира, д. 9а",53.316068,50.303123,10:00:00,22:00:00,None,None
1049,Егорьевск,"140301, Егорьевск, Советская д.113",55.381838,39.038420,08:00:00,23:00:00,09:00:00,11:00:00
1050,Химки,"141425, Химки, Международный Аэропорт Шереметь...",55.963552,37.414254,07:30:00,23:00:00,None,None


## Узнаем подходящие адреса ресторанов:

In [16]:
requare_time = '08:45:00'

requare_city = 'Новосибирск'

conn = sl.connect("kfc.db")

cursor = conn.cursor()

query = """ SELECT * FROM restaurants WHERE city = '{0}'
            AND time(breakfast_start_time) <= time('{1}') 
            AND time(breakfast_end_time) >= time('{1}')
            AND breakfast_start_time NOT IN('')""".format(requare_city, requare_time) 

df = pd.read_sql(query, conn)

conn.close()

df

,city,address,latitude,longitude,start_time,end_time,breakfast_start_time,breakfast_end_time
0,Новосибирск,"630099, Новосибирск, Красный проспект, 29",55.031273,82.919411,00:00:00,23:59:00,08:00:00,11:00:00
1,Новосибирск,"630078, Новосибирск, площадь Карла Маркса, 2",54.983633,82.892683,08:00:00,23:59:59,08:00:00,11:00:00
2,Новосибирск,"630087, Новосибирск, проспект Карла Маркса, 24",54.989713,82.908052,08:00:00,22:55:00,08:00:00,11:00:00
3,Новосибирск,"630082, Новосибирск, Дуси Ковальчук, 179/4",55.060571,82.914217,00:00:00,23:59:59,07:00:00,11:00:00
4,Новосибирск,"630107, Новосибирск, Троллейная, 130",54.964728,82.853306,08:00:00,23:59:59,08:00:00,11:00:00
5,Новосибирск,"630108, Новосибирск, проезд Энергетиков, 9",55.009649,82.877532,05:00:00,04:00:00,06:00:00,11:00:00
6,Новосибирск,"630129, Новосибирск, Курчатова, 1",55.104311,82.960692,10:00:00,22:00:00,08:00:00,11:00:00
7,Новосибирск,"630108, Новосибирск, Станционная, 32",54.997494,82.851273,08:00:00,23:59:59,08:00:00,11:00:00
8,Новосибирск,"630009, Новосибирск, Большевистская, 12/1",55.007276,82.937683,09:00:00,22:00:00,08:00:00,11:00:00
9,Новосибирск,"630017, Новосибирск, Военная, 7/1",55.027175,82.938540,08:00:00,02:00:00,08:00:00,11:00:00
